<h1 style="color:#6B7A8F;"><strong>MOBA Chat Moderator</strong></h1>

<h3 style="color:#6B7A8F;"><strong>Cleaning and Processing</strong></h3>

In [1]:
# --- Librerías ---
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath('../src'))
from cleaning import inspect_dataframe, apply_binary_mapping, apply_multi_mapping, clean_text, apply_manual_toxic_overrides

# --- Cargar los DataFrames ---
df_conda_train = pd.read_csv("../data/raw/CONDA_train.csv")
df_conda_val = pd.read_csv("../data/raw/CONDA_valid.csv")
df_conda_test = pd.read_csv("../data/raw/CONDA_test.csv")

df_hf_train = pd.read_csv("../data/raw/train.csv")
df_hf_val = pd.read_csv("../data/raw/validation.csv")
df_hf_test = pd.read_csv("../data/raw/test.csv")

In [2]:
# --- Inspección ---
inspect_dataframe(df_conda_train, "CONDA_train")
inspect_dataframe(df_conda_val, "CONDA_validation")
inspect_dataframe(df_conda_test, "CONDA_test")

inspect_dataframe(df_hf_train, "HuggingFace_train")
inspect_dataframe(df_hf_val, "HuggingFace_validation")
inspect_dataframe(df_hf_test, "HuggingFace_test")


--- Inspección de: CONDA_train ---

>>> Primeras 5 filas:
      Id  matchId  conversationId utterance  chatTime  playerSlot  \
0  11263      697            3193      wow!        76           0   
1  13741      843            3809       WTF      1563           5   
2  22125     1412            6199   wpe wpe      2853           1   
3   6453      439            1875    hahaha      1038           0   
4   9644      601            2713       wtf      1661           5   

                  playerId intentClass slotClasses          slotTokens  
0  ANTS IN MY EYES JOHNSON           O          O            wow (O),   
1                      M.k           O          T            WTF (T),   
2              Acqua Ragia           O        O O   wpe (O), wpe (O),   
3                 juicebox           O          O         hahaha (O),   
4            KAIST.Shadows           O          T            wtf (T),   

>>> Últimas 5 filas:
          Id  matchId  conversationId               utterance  cha

In [3]:
# --- Limpiamos test primero ---
# Nos quedamos solo con los comentarios de test porque luego vamos a usarlo como CSV de prueba para la aplicación.
df_conda_test = df_conda_test.rename(columns={'utterance': 'text'})
prueba_aplicacion = df_conda_test[['text']].copy()

# --- Guardamos ---
#raw_test_for_app_path = '../data/processed/prueba_aplicacion.csv'
#os.makedirs(os.path.dirname(raw_test_for_app_path), exist_ok=True)
#prueba_aplicacion.to_csv(raw_test_for_app_path, index=False)

In [4]:
# --- Concatenamos df ---
df_conda = pd.concat([df_conda_train, df_conda_val], axis=0)
df_hf = pd.concat([df_hf_test,df_hf_train,df_hf_val], axis= 0)

# --- Limpiamos df independientemente ---
df_hf = df_hf.rename(columns={'message': 'text'})
df_hf = df_hf.rename(columns={'target': 'original_label'})
df_hf = apply_binary_mapping(df_hf, 'hf')
df_hf = apply_multi_mapping(df_hf, 'hf')
df_hf = df_hf[['text', 'original_label', 'binary_label', 'multi_label']].copy()

df_conda = df_conda.rename(columns={'utterance': 'text'})
df_conda = df_conda.rename(columns={'intentClass': 'original_label'})
df_conda = apply_binary_mapping(df_conda, 'conda')
df_conda = apply_multi_mapping(df_conda, 'conda')
df_conda = df_conda[['text', 'original_label', 'binary_label', 'multi_label']].copy()

# --- Unimos datasets ---
df_final = pd.concat([df_conda, df_hf], axis=0, ignore_index= True)

# --- Limpiamos text ---
df_final['text_cleaned'] = df_final['text'].apply(clean_text)

# --- Ajustamos las clasificaciones ---
# Dentro de la clasificación todavia habia algunos que ajustar porque quedaban como no toxicos.
df_final = apply_manual_toxic_overrides(df_final)

# --- Guardamos ---
#processed_data_path = '../data/processed/df_final.csv'
#os.makedirs(os.path.dirname(processed_data_path), exist_ok=True)
#df_final.to_csv(processed_data_path, index=False)